```{contents}
```
## Model Selection

**Model selection** in LangGraph is the systematic process of **choosing, routing, and coordinating LLMs** inside a graph so that each step of a workflow uses the **most appropriate model** for its task, subject to constraints on **cost, latency, quality, safety, and reliability**.

Unlike simple applications that bind a single model globally, LangGraph treats models as **first-class, dynamic execution resources**.

---

### **1. Why Model Selection Matters**

LLM workloads are **heterogeneous**:

| Task Type        | Optimal Model Properties |
| ---------------- | ------------------------ |
| Planning         | High reasoning, slower   |
| Tool execution   | Cheap, fast              |
| Verification     | Deterministic, stable    |
| Draft writing    | Creative, expressive     |
| Classification   | Small, low latency       |
| Safety filtering | Guard model              |

Using one model for everything leads to **wasted cost, slow latency, and poor reliability**.

LangGraph allows **per-node, per-decision, runtime model choice**.

---

### **2. Where Model Selection Lives in LangGraph**

Model selection occurs at **node boundaries**:

```
State → [Router Node] → { Model A | Model B | Model C } → Next State
```

Selection is driven by **state**, **context**, and **policy**.

---

### **3. Model Binding Strategies**

| Strategy              | Description                        |
| --------------------- | ---------------------------------- |
| Static binding        | Model fixed at node definition     |
| Dynamic binding       | Model chosen at runtime            |
| Hierarchical routing  | Supervisor chooses model           |
| Fallback routing      | Switch on failure                  |
| Cost-aware routing    | Choose cheapest that meets quality |
| Latency-aware routing | Choose fastest under deadline      |
| Risk-aware routing    | Choose safest under uncertainty    |

---

### **4. Core Mechanism: Model Router Node**

```python
def choose_model(state):
    if state["task"] == "planning":
        return "gpt-4o"
    if state["task"] == "classification":
        return "gpt-3.5-turbo"
    return "gpt-4o-mini"
```

```python
builder.add_conditional_edges(
    "router",
    choose_model,
    {
        "gpt-4o": "planner_node",
        "gpt-3.5-turbo": "classifier_node",
        "gpt-4o-mini": "fast_node"
    }
)
```

---

### **5. Per-Node Model Execution**

```python
from langchain_openai import ChatOpenAI

planner = ChatOpenAI(model="gpt-4o")
fast = ChatOpenAI(model="gpt-4o-mini")
cheap = ChatOpenAI(model="gpt-3.5-turbo")
```

Each node binds its own model:

```python
def planner_node(state):
    return {"plan": planner.invoke(state["input"]).content}
```

---

### **6. Cost–Quality Optimization Pattern**

| Phase   | Model        |
| ------- | ------------ |
| Plan    | Large model  |
| Execute | Small model  |
| Verify  | Medium model |
| Refine  | Large model  |

This reduces total cost by **40–70%** in real deployments.

---

### **7. Fallback & Recovery**

```python
def robust_llm_call(prompt):
    try:
        return strong_model.invoke(prompt)
    except:
        return cheap_model.invoke(prompt)
```

Used inside a node to guarantee availability.

---

### **8. Model Selection Variants**

| Variant              | Use Case           |
| -------------------- | ------------------ |
| Single-model graph   | Prototypes         |
| Multi-model pipeline | Production         |
| Self-adaptive models | Autonomous systems |
| Budget-aware routing | SaaS platforms     |
| Safety-tier routing  | Regulated domains  |

---

### **9. Production Policies**

| Policy            | Enforced Through  |
| ----------------- | ----------------- |
| Max cost per run  | Router logic      |
| Latency SLA       | Router + timeouts |
| Safety compliance | Guard model       |
| Model versioning  | Config management |
| Canary deployment | Weighted routing  |

---

### **10. Mental Model**

LangGraph turns model usage from:

> **Static dependency** → **Dynamic execution resource**

allowing systems to behave like **intelligent schedulers of cognition**.



### Demonstration

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_openai import ChatOpenAI

# -----------------------------
# State Schema
# -----------------------------
class State(TypedDict):
    task: str
    input: str
    output: str

# -----------------------------
# Models
# -----------------------------
planner_model = ChatOpenAI(model="gpt-4o")
fast_model = ChatOpenAI(model="gpt-4o-mini")
cheap_model = ChatOpenAI(model="gpt-3.5-turbo")

# -----------------------------
# Nodes
# -----------------------------
def router(state: State):
    if state["task"] == "planning":
        return "planner"
    if state["task"] == "classification":
        return "cheap"
    return "fast"

def planner_node(state: State):
    result = planner_model.invoke(state["input"]).content
    return {"output": result}

def fast_node(state: State):
    result = fast_model.invoke(state["input"]).content
    return {"output": result}

def cheap_node(state: State):
    result = cheap_model.invoke(state["input"]).content
    return {"output": result}

# -----------------------------
# Build Graph
# -----------------------------
builder = StateGraph(State)

builder.add_node("router", lambda s: s)
builder.add_node("planner", planner_node)
builder.add_node("fast", fast_node)
builder.add_node("cheap", cheap_node)

builder.set_entry_point("router")

builder.add_conditional_edges(
    "router",
    router,
    {
        "planner": "planner",
        "fast": "fast",
        "cheap": "cheap",
    }
)

builder.add_edge("planner", END)
builder.add_edge("fast", END)
builder.add_edge("cheap", END)

graph = builder.compile()

# -----------------------------
# Run
# -----------------------------
result = graph.invoke({
    "task": "planning",
    "input": "Design a 7-day fitness plan for beginners"
})

print(result["output"])


Creating a 7-day fitness plan for beginners involves balancing different types of exercises to ensure a comprehensive approach that addresses cardiovascular fitness, strength, flexibility, and recovery. Here's a suggested plan:

### Day 1: Cardio & Core
- **Warm-up:** 5-10 minutes of light jogging or brisk walking.
- **Workout:**
  - 20 minutes of steady-state cardio (e.g., brisk walking, cycling, or elliptical)
  - Core exercises:
    - Plank: 3 x 30 seconds
    - Bicycle crunches: 3 x 15 reps
    - Leg raises: 3 x 10 reps
- **Cool down:** 5 minutes of stretching focusing on the core and lower back.

### Day 2: Upper Body Strength
- **Warm-up:** 5-10 minutes of arm circles, shoulder rolls, and light cardio.
- **Workout:**
  - Push-ups (knee or standard): 3 x 8-10 reps
  - Dumbbell shoulder press: 3 x 8-10 reps
  - Dumbbell rows: 3 x 8-10 reps per arm
  - Triceps dips (using a chair or bench): 3 x 8-10 reps
- **Cool down:** 5-10 minutes of stretching focusing on the arms, shoulders, an